## Great, now that we discussed a little let's continue

Given that the current approach utilized by the authors lacks reproducibility, we will explore an alternative method by leveraging nf-core pipelines for data analysis.

Please explain, how we will achieve reproducibility for the course  with this approach.


Using nf-core pipelines ensures reproducibility mainly because the workflow is standardized by container, tool and library versions.

You have successfully downloaded 2 of the fastq files we will use in our study.

What is the next step if we want to first have a count table and check the quality of our fastq files? What is the pipeline called to do so?

I would run the pipeline nf-core/rnaseq on the downloaded files, which gives a count table that is ready for DE analysis and a QC report for each file.

Analyze the 2 files using an nf-core pipeline.

What does this pipeline do?

Which are the main tools that will be used in the pipeline?

The pipeline does Merging of the files (Preprocessing with QC), Alignment with Quantification (Genome: STAR/HISAT, Pseudo: Salmon/Kalisco), Post Processing and final QC with an overall report. In detail as described in the intro of the pipeline:

1. Merge re-sequenced FastQ files (cat)
2. Auto-infer strandedness by subsampling and pseudoalignment (fq, Salmon)
3. Read QC (FastQC)
4. UMI extraction (UMI-tools)
5. Adapter and quality trimming (Trim Galore!)
6. Removal of genome contaminants (BBSplit)
7. Removal of ribosomal RNA (SortMeRNA)
8. Choice of multiple alignment and quantification routes (For STAR the sentieon implementation can be chosen):
  - STAR -> Salmon
  - STAR -> RSEM
  - HiSAT2 -> NO QUANTIFICATION
9. Sort and index alignments (SAMtools)
10. UMI-based deduplication (UMI-tools)
11. Duplicate read marking (picard MarkDuplicates)
12. Transcript assembly and quantification (StringTie)
13. Create bigWig coverage files (BEDTools, bedGraphToBigWig)
14. Extensive quality control:
  - RSeQC
  - Qualimap
  - dupRadar
  - Preseq
  - DESeq2
  - Kraken2 -> Bracken on unaligned sequences; optional
15. Pseudoalignment and quantification (Salmon or ‘Kallisto’; optional)
16. Present QC for raw read, alignment, gene biotype, sample similarity, and strand-specificity checks (MultiQC, R)

As all other nf-core pipelines, the chosen pipeline takes in a samplesheet as input.

Use Python and pandas to create the samplesheet for your 2 samples. Feel free to make use of the table you created earlier today.

Choose your sample names wisely, they must be the connection of the results to the metadata. If you can't find the sample in the metadata later, the analysis was useless.

In [1]:
import pandas as pd

samples_info = {
    "Run": ["SRR23195516", "SRR23195511"],
    "SNI": [True, False],
    "Oxy": [True, True],
    "strandedness": 'auto',
    "Bases": [6922564500, 6456390900],

    "fastq_1": [
        "./expression_data/fastq/SRX19144486_SRR23195516_1.fastq.gz",
        "./expression_data/fastq/SRX19144488_SRR23195511_1.fastq.gz"
    ],
    "fastq_2": [
        "./expression_data/fastq/SRX19144486_SRR23195516_2.fastq.gz",
        "./expression_data/fastq/SRX19144488_SRR23195511_2.fastq.gz"
    ]
}

samplesheet = pd.DataFrame(samples_info)
samplesheet["sample"] = samplesheet.apply(lambda x: f"{x['Run']}_SNI{x['SNI']}_Oxy{x['Oxy']}", axis=1)
samplesheet = samplesheet[["sample", "fastq_1", "fastq_2", "strandedness", "SNI", "Oxy", "Bases"]]
samplesheet.to_csv("samplesheet_2_samples.csv", index=False)
samplesheet


,sample,fastq_1,fastq_2,strandedness,SNI,Oxy,Bases
0,SRR23195516_SNITrue_OxyTrue,./expression_data/fastq/SRX19144486_SRR2319551...,./expression_data/fastq/SRX19144486_SRR2319551...,auto,True,True,6922564500
1,SRR23195511_SNIFalse_OxyTrue,./expression_data/fastq/SRX19144488_SRR2319551...,./expression_data/fastq/SRX19144488_SRR2319551...,auto,False,True,6456390900


In [ ]:
!nextflow run nf-core/rnaseq \
  -profile docker \
  --input samplesheet_2_samples.csv \
  --outdir processed_expression_data \
  --genome GRCm38 \
  --paired_end \
  -c nextflow.config \
  -resume



 N E X T F L O W   ~  version 25.04.7

Launching `https://github.com/nf-core/rnaseq` [exotic_heyrovsky] DSL2 - revision: 9738a2df42 [master]


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/rnaseq 3.21.0
------------------------------------------------------
Input/output options
  input              : samplesheet_2_samples.csv
  outdir             : processed_expression_data

Reference genome options
  genome             : mm10
  fasta              : s3://ngi-igenomes/igenomes//Mus_musculus/UCSC/mm10/Sequence/WholeGenomeFasta/genome.fa
  gtf                : s3://ngi-igenomes/igenomes//Mus_musculus/UCSC/mm10/Annotation/Genes/genes.gtf
  gene_bed           : s3://ngi-igenomes/igenomes//Mus_musculus/UCSC/mm10/Annotation/Genes/ge

Explain all the parameters you set and why you set them in this way.



1. --input samplesheet_2_samples.csv -> Input samplesheet that is reordered and connected with metadata
2. --outdir processed_expression_data -> Directory for results
3. --genome mm10 -> The reference genome we are using for the mapping
4. --paired_end -> Use the pairwise read style
5. -c nextflow.config -> Dont run into issues with download of the genome -> Defined max CPU and max Menory for critical processes

## Browsing the results

How did the pipeline perform?

TODO

Explain the quality control steps. Are you happy with the quality and why. If not, why not.
Please give additional information on : 
- ribosomal rRNA
- Duplication
- GC content

What are the possible steps that could lead to poorer results?

TODO

Would you exclude any samples? If yes, which and why?

In [ ]:
# TODO

What would you now do to continue the experiment? What are the scientists trying to figure out? Which packages on R or python would you use?

I would do DE analysis next. The authers want to figure out the connection between oxycodone withdrawal and pain on a transcriptimical level, so we need to find significant DE genes between groups (e.g. DE in Oxy vs. Sal in SNI and Sham) and then compare these sets to see the difference of the oxy withdrawal between SNI and Sham. I would use the packages for venn diagramms and heatmaps (seaborn) to compare the visusalization of my data with the visusalization of theri data.